In [ ]:
import tensorflow as tf
import keras
from keras import layers
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_dir = '/content/drive/MyDrive/CatClassifier/train'

In [ ]:
train_datagen = ImageDataGenerator(
    validation_split=0.2,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    rotation_range=15,)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=32,
    subset='training',
    class_mode='binary',
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=32,
    subset='validation',
    class_mode='binary'
)

Found 20001 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [ ]:
# 추가 학습을 위해 미리 저장한 모델을 불러옴
additional_model = keras.models.load_model('/content/drive/MyDrive/CatClassifier/GoogLeNet')

In [ ]:
additional_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_59 (Conv2D)              (None, 112, 112, 64) 9472        input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_13 (MaxPooling2D) (None, 56, 56, 64)   0           conv2d_59[0][0]                  
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 56, 56, 64)   0           max_pooling2d_13[0][0]           
______________________________________________________________________________________________

In [ ]:
additional_model.compile(optimizer=keras.optimizers.SGD(lr=0.001, momentum=0.9, decay = 0.0005),
              loss={'main_classifier' : 'sparse_categorical_crossentropy',
                    'auxiliary_4a' : 'sparse_categorical_crossentropy',
                    'auxiliary_4d' : 'sparse_categorical_crossentropy'},
                loss_weights={'main_classifier' : 1.0,
                              'auxiliary_4a' : 0.3,
                              'auxiliary_4d' : 0.3},
                metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
checkpoint = ModelCheckpoint(filepath='/content/drive/MyDrive/CatClassifier/GoogLeNet',
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             save_weights_only=False,
                             mode='auto',)

early = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience = 5, verbose = 1, mode= 'auto')

In [ ]:
num_train_samples = train_generator.n
num_train_batches = train_generator.batch_size
print(num_train_samples,' , ',num_train_batches)
train_size = num_train_samples // num_train_batches
print(train_size)

20001  ,  32
625


In [ ]:
num_val_samples = validation_generator.n
num_val_batches = validation_generator.batch_size
print(num_val_samples,' , ',num_val_batches)
val_size = num_val_samples // num_val_batches
print(val_size)

5000  ,  32
156


In [ ]:
hist = additional_model.fit(train_generator,
                 steps_per_epoch=train_size, 
                 epochs = 100 , 
                 validation_data=validation_generator ,
                 validation_steps=val_size ,
                 verbose=1,
                 callbacks=[checkpoint, early])

Epoch 1/100
625/625 [==============================] - 15621s 25s/step - loss: 1.0509 - main_classifier_loss: 0.6580 - auxiliary_4a_loss: 0.6545 - auxiliary_4d_loss: 0.6550 - main_classifier_accuracy: 0.6086 - auxiliary_4a_accuracy: 0.6120 - auxiliary_4d_accuracy: 0.6148 - val_loss: 0.9853 - val_main_classifier_loss: 0.6132 - val_auxiliary_4a_loss: 0.6247 - val_auxiliary_4d_loss: 0.6154 - val_main_classifier_accuracy: 0.6685 - val_auxiliary_4a_accuracy: 0.6671 - val_auxiliary_4d_accuracy: 0.6701

Epoch 00001: val_loss improved from inf to 0.98529, saving model to /content/drive/MyDrive/CatClassifier/GoogLeNet
INFO:tensorflow:Assets written to: /content/drive/MyDrive/CatClassifier/GoogLeNet/assets
Epoch 2/100
625/625 [==============================] - 352s 564ms/step - loss: 1.0249 - main_classifier_loss: 0.6415 - auxiliary_4a_loss: 0.6397 - auxiliary_4d_loss: 0.6384 - main_classifier_accuracy: 0.6320 - auxiliary_4a_accuracy: 0.6405 - auxiliary_4d_accuracy: 0.6400 - val_loss: 1.0090 - v

In [ ]:
additional_model.evaluate(train_generator)

626/626 [==============================] - 255s 407ms/step - loss: 0.4875 - main_classifier_loss: 0.2942 - auxiliary_4a_loss: 0.3439 - auxiliary_4d_loss: 0.3007 - main_classifier_accuracy: 0.8732 - auxiliary_4a_accuracy: 0.8569 - auxiliary_4d_accuracy: 0.8709


[0.48754921555519104,
 0.29417383670806885,
 0.3439052104949951,
 0.30067914724349976,
 0.8731563687324524,
 0.8569071292877197,
 0.8709064722061157]

In [ ]:
model.save('/content/drive/MyDrive/CatClassifier/GoogLeNet/additional train (2)')